In [1]:
from tqdm import tqdm
import requests
import pandas as pd
import time
from bs4 import BeautifulSoup
import os

In [13]:
movers_to_scrape_metadata_for = pd.read_csv(
    "movers_2021-2024.txt",
    header=None,
    delimiter=";",
    names=["totas_id", "mover_id", "label"],
)
movers_to_scrape_metadata_for["label"].unique()

array([1, 2, 3])

In [14]:
movers_to_scrape_metadata_for[movers_to_scrape_metadata_for["label"] == 3]

,totas_id,mover_id,label
3244,507665,M315765,3
3890,508311,M316000,3
4077,508498,M316125,3
4331,508753,M316236,3
4341,508763,M316243,3
...,...,...,...
41798,546251,M332146,3
41828,546281,M332169,3
41919,546372,M332232,3
41988,546441,M332291,3


In [16]:
label_conversion = {1: 1, 2: 0, 3: -1}  # real  # bogus  # unknown

In [17]:
movers_to_scrape_metadata_for["label"] = movers_to_scrape_metadata_for["label"].map(
    label_conversion
)
movers_to_scrape_metadata_for

,totas_id,mover_id,label
0,504421,TKCCA25,1
1,504422,TKCCA26,1
2,504423,TKCCA27,1
3,504424,TKCCA28,1
4,504425,TKCCA29,1
...,...,...,...
43068,547521,TO2C148,1
43069,547522,TO2C149,1
43070,547523,TO2C150,1
43071,547524,TO2C151,1


In [18]:
movers = movers_to_scrape_metadata_for[movers_to_scrape_metadata_for["label"] != -1]

In [21]:
movers

,totas_id,mover_id,label
0,504421,TKCCA25,1
1,504422,TKCCA26,1
2,504423,TKCCA27,1
3,504424,TKCCA28,1
4,504425,TKCCA29,1
...,...,...,...
43068,547521,TO2C148,1
43069,547522,TO2C149,1
43070,547523,TO2C150,1
43071,547524,TO2C151,1


In [43]:
movers["totas_id"].unique()

array([504421, 504422, 504423, ..., 547523, 547524, 547525])

In [41]:
# save the data
movers.to_csv("movers_2021-2024_cleaned.txt", index=False)

In [23]:
base = "https://totas.cosmos.esa.int/mover.php?id="
real_test = 504423
bogus_test = 504485

In [24]:
real_url = base + str(real_test)
bogus_url = base + str(bogus_test)

In [26]:
real_request = requests.get(real_url)
real_soup = BeautifulSoup(real_request.text, "html.parser")

In [37]:
bogus_request = requests.get(bogus_url)
bogus_soup = BeautifulSoup(bogus_request.text, "html.parser")

In [39]:
images = bogus_soup.findAll("img")
images = [
    image["src"] for image in images if "b" in image["src"] and ".png" in image["src"]
]
images = list(set(images))
images

['image-view.php?file=2021/02/08/026036/movers/b0504485-2.png',
 'image-view.php?file=2021/02/08/026036/movers/b0504485-3.png',
 'image-view.php?file=2021/02/08/026036/movers/b0504485-4.png',
 'image-view.php?file=2021/02/08/026036/movers/b0504485-1.png']

In [40]:
images = [image.split("/")[-1] for image in images]
images

['b0504485-2.png', 'b0504485-3.png', 'b0504485-4.png', 'b0504485-1.png']

In [ ]:
def get_mover_to_image_mapping(mover_id):
    base = "https://totas.cosmos.esa.int/mover.php?id="
    url = base + str(mover_id)
    request = requests.get(url)
    soup = BeautifulSoup(request.text, "html.parser")
    images = soup.findAll("img")
    images = [
        image["src"]
        for image in images
        if "b" in image["src"] and ".png" in image["src"]
    ]
    images = list(set(images))
    images = [image.split("/")[-1] for image in images]

    return images